# Probabilities through monte carlo simulations

Here, rather than training a model to predict the $E[\text{dice roll score}]$, I just build a monte carlo simulation. 

In [1]:
import sys
sys.path.insert(0, '../../src')

In [2]:
import json
import random
from time import perf_counter
from collections import defaultdict

from farkle.logic import gameobjects as go

In [3]:
dh = go.DiceHand(2,3)
dh.possible_scores()

[]

In [4]:
# for saving dicehands as json
class DiceHandEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, go.DiceHand):
            return obj.json_encode
        return super().default(obj)
    
def as_dicehand(dct):
    if '__DiceHand__' in dct:
        return go.DiceHand.json_decode(dct)
    return dct

In [5]:
j = json.dumps({'dh': dh}, cls=DiceHandEncoder)

In [6]:
j

'{"dh": {"__DiceHand__": true, "free": [2, 3], "locked": [], "score": 0}}'

In [7]:
jl = json.loads(j, object_hook=as_dicehand)

In [8]:
jl

{'dh': DiceHand(free=[2, 3], locked=[], score=0)}

In [19]:
# cant use dicehand as key even with encoder
# maybe use the hash? 
j = json.dumps({dh: 5}, cls=DiceHandEncoder)

TypeError: keys must be str, int, float, bool or None, not DiceHand

In [20]:
hash(dh)

-1025832054940974799

In [40]:
type(hash(dh))

int

In [6]:
j

'{"dh": {"__DiceHand__": true, "free": [2, 3], "locked": [], "score": 0}}'

In [7]:
jl = json.loads(j, object_hook=as_dicehand)

In [8]:
jl

{'dh': DiceHand(free=[2, 3], locked=[], score=0)}

In [9]:
type(jl['dh'])

farkle.logic.gameobjects.DiceHand

In [10]:
d = {0: 'a', 1: 'b'}

In [11]:
str(d)

"{0: 'a', 1: 'b'}"

In [12]:
print(d)

{0: 'a', 1: 'b'}


In [113]:
class MonteCarloSimulation(object):
    def __init__(self, max_dice: int = 6):
        self.max_dice = max_dice
        self.dice_range = range(1, self.max_dice + 1)
        self.dice_hands = {i: go.DiceHand(num_dice=i) for i in self.dice_range}
        
        self.roll_obs = {i: 0 for i in self.dice_range}
        self.possible_score_obs = {i: 0 for i in self.dice_range}
        self.possible_score_frequencies = {i: defaultdict(int) for i in self.dice_range}
        
    def roll(self, num_dice: int):
        self.dice_hands[num_dice].roll()
        self.roll_obs[num_dice] += 1
        if self.dice_hands[num_dice].possible_scores():
            for dh in self.dice_hands[num_dice].possible_scores():
                dh: go.DiceHand
                self.possible_score_obs[num_dice] += 1
                self.possible_score_frequencies[num_dice][dh] += 1
        else:
            self.possible_score_frequencies[num_dice][None] += 1
            
    def roll_all(self, num_rolls: int = 1):
        """Rolls all hands in ratio where 1 dice is 1 roll and 
        each additional die is 6x more rolls"""
        for die_num in self.dice_range:
            for roll_num in range(6**(die_num-1)):
                self.roll(die_num)
    
    def P_farkle(self, num_dice: int):
        return self.possible_score_frequencies[num_dice][None] / self.roll_obs[num_dice]
    
    def E_score(self, num_dice: int):
        E_score = 0
        for dh, freq in self.possible_score_frequencies[num_dice].items():
            if dh: E_score += dh.score * freq
        return (E_score / self.possible_score_obs[num_dice]) * (1 - self.P_farkle(num_dice))
    
    # todo: probs best as an iter
    @property
    def _hash_frequencies(self):
        hash_to_dh = {}
        hash_to_freq = {}
        for d in self.dice_range:
            hash_to_freq[d] = {}
            for dh, freq in self.possible_score_frequencies[d].items():
                hash_to_dh[str(hash(dh))] = dh
                hash_to_freq[d][str(hash(dh))] = freq
        return hash_to_dh, hash_to_freq
    
    @staticmethod
    def _unhash_frequencies(hash_to_dh, hash_to_freq):
        _possible_score_freq = {}
        for d in hash_to_freq:
            d_int = int(d)
            _possible_score_freq[d_int] = {}
            for h, freq in hash_to_freq[d].items():
                _possible_score_freq[d_int][hash_to_dh[h]] = int(freq)
        return _possible_score_freq
        
    @property
    def json_encode(self):
        hash_to_dh, hash_to_freq = self._hash_frequencies
        d = {'max_dice': self.max_dice, 
             'roll_obs': self.roll_obs, 
             'possible_score_obs': self.possible_score_obs, 
             'hash_to_dh': hash_to_dh, 
             'hash_to_freq': hash_to_freq}
             # 'possible_score_frequencies': self.possible_score_frequencies}
        return json.dumps(d, cls=DiceHandEncoder)
    
    def save(self, path: str):
        j = self.json_encode
        with open(path, 'w') as f:
            f.write(j)
        
    @staticmethod
    def load(path: str):
        with open(path, 'r') as f:
            j = json.loads(f.read(), object_hook=as_dicehand)
        
        mcs = MonteCarloSimulation(j['max_dice'])
        mcs.roll_obs = j['roll_obs']
        mcs.possible_score_obs = j['possible_score_obs']
        mcs.possible_score_frequencies = MonteCarloSimulation._unhash_frequencies(j['hash_to_dh'], 
                                                                                  j['hash_to_freq'])
        return mcs

In [101]:
mcs = MonteCarloSimulation()

In [102]:
mcs.dice_hands[6]

DiceHand(free=[1, 1, 1, 3, 5, 6], locked=[], score=0)

In [103]:
for i in range(20):
    mcs.roll(6)

In [104]:
mcs.possible_score_frequencies

{1: defaultdict(int, {}),
 2: defaultdict(int, {}),
 3: defaultdict(int, {}),
 4: defaultdict(int, {}),
 5: defaultdict(int, {}),
 6: defaultdict(int,
             {DiceHand(free=[6, 6, 6], locked=[], score=600): 3,
              DiceHand(free=[1], locked=[], score=100): 15,
              DiceHand(free=[1, 5], locked=[], score=150): 13,
              DiceHand(free=[1, 5, 5], locked=[], score=200): 5,
              DiceHand(free=[5], locked=[], score=50): 16,
              DiceHand(free=[5, 5], locked=[], score=100): 5,
              DiceHand(free=[1, 1], locked=[], score=200): 4,
              DiceHand(free=[1, 1, 5], locked=[], score=250): 4,
              DiceHand(free=[1, 1, 5, 5], locked=[], score=300): 2,
              None: 1,
              DiceHand(free=[1, 4, 4, 4], locked=[], score=500): 1,
              DiceHand(free=[4, 4, 4], locked=[], score=400): 1,
              DiceHand(free=[1, 6, 6, 6], locked=[], score=700): 2,
              DiceHand(free=[1, 5, 6, 6, 6], locked=[], 

In [105]:
h, f = mcs._hash_frequencies

In [106]:
h

{'5084703364844673703': DiceHand(free=[6, 6, 6], locked=[], score=600),
 '7632524756553394483': DiceHand(free=[1], locked=[], score=100),
 '6209970323174613410': DiceHand(free=[1, 5], locked=[], score=150),
 '2390374266854751092': DiceHand(free=[1, 5, 5], locked=[], score=200),
 '6457213122273063421': DiceHand(free=[5], locked=[], score=50),
 '5353976940633003237': DiceHand(free=[5, 5], locked=[], score=100),
 '-1809235098275554318': DiceHand(free=[1, 1], locked=[], score=200),
 '-7810874240232125281': DiceHand(free=[1, 1, 5], locked=[], score=250),
 '-2902616736069918135': DiceHand(free=[1, 1, 5, 5], locked=[], score=300),
 '271773578': None,
 '-1520611015974714827': DiceHand(free=[1, 4, 4, 4], locked=[], score=500),
 '-5490153455905078781': DiceHand(free=[4, 4, 4], locked=[], score=400),
 '-176996419478431460': DiceHand(free=[1, 6, 6, 6], locked=[], score=700),
 '-6971664803686750677': DiceHand(free=[1, 5, 6, 6, 6], locked=[], score=750),
 '3573966809515336244': DiceHand(free=[5, 6, 

In [107]:
f

{1: {},
 2: {},
 3: {},
 4: {},
 5: {},
 6: {'5084703364844673703': 3,
  '7632524756553394483': 15,
  '6209970323174613410': 13,
  '2390374266854751092': 5,
  '6457213122273063421': 16,
  '5353976940633003237': 5,
  '-1809235098275554318': 4,
  '-7810874240232125281': 4,
  '-2902616736069918135': 2,
  '271773578': 1,
  '-1520611015974714827': 1,
  '-5490153455905078781': 1,
  '-176996419478431460': 2,
  '-6971664803686750677': 1,
  '3573966809515336244': 1,
  '-8432349949530499542': 1,
  '-203550692328843228': 1}}

In [108]:
mcs.json_encode

'{"max_dice": 6, "roll_obs": {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0, "6": 20}, "possible_score_obs": {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0, "6": 75}, "hash_to_dh": {"5084703364844673703": {"__DiceHand__": true, "free": [6, 6, 6], "locked": [], "score": 600}, "7632524756553394483": {"__DiceHand__": true, "free": [1], "locked": [], "score": 100}, "6209970323174613410": {"__DiceHand__": true, "free": [1, 5], "locked": [], "score": 150}, "2390374266854751092": {"__DiceHand__": true, "free": [1, 5, 5], "locked": [], "score": 200}, "6457213122273063421": {"__DiceHand__": true, "free": [5], "locked": [], "score": 50}, "5353976940633003237": {"__DiceHand__": true, "free": [5, 5], "locked": [], "score": 100}, "-1809235098275554318": {"__DiceHand__": true, "free": [1, 1], "locked": [], "score": 200}, "-7810874240232125281": {"__DiceHand__": true, "free": [1, 1, 5], "locked": [], "score": 250}, "-2902616736069918135": {"__DiceHand__": true, "free": [1, 1, 5, 5], "locked": [], "score": 300}, "2

In [109]:
mcs.save('test_export.json')

In [114]:
mcs_loaded = MonteCarloSimulation.load('test_export.json')

In [117]:
mcs_loaded.possible_score_frequencies

{1: {},
 2: {},
 3: {},
 4: {},
 5: {},
 6: {DiceHand(free=[6, 6, 6], locked=[], score=600): 3,
  DiceHand(free=[1], locked=[], score=100): 15,
  DiceHand(free=[1, 5], locked=[], score=150): 13,
  DiceHand(free=[1, 5, 5], locked=[], score=200): 5,
  DiceHand(free=[5], locked=[], score=50): 16,
  DiceHand(free=[5, 5], locked=[], score=100): 5,
  DiceHand(free=[1, 1], locked=[], score=200): 4,
  DiceHand(free=[1, 1, 5], locked=[], score=250): 4,
  DiceHand(free=[1, 1, 5, 5], locked=[], score=300): 2,
  None: 1,
  DiceHand(free=[1, 4, 4, 4], locked=[], score=500): 1,
  DiceHand(free=[4, 4, 4], locked=[], score=400): 1,
  DiceHand(free=[1, 6, 6, 6], locked=[], score=700): 2,
  DiceHand(free=[1, 5, 6, 6, 6], locked=[], score=750): 1,
  DiceHand(free=[5, 6, 6, 6], locked=[], score=650): 1,
  DiceHand(free=[3, 3, 3, 5], locked=[], score=350): 1,
  DiceHand(free=[3, 3, 3], locked=[], score=300): 1}}

In [118]:
mcs_loaded.possible_score_frequencies[6][go.DiceHand(1, score=100)]

15

In [22]:
mcs.dice_hands[6]

DiceHand(6, 6, 2, 6, 5, 6, score=0)

In [25]:
t0 = perf_counter()
for i in range(6**5):
    if i % 300 == 0:
        print(i)
    mcs.roll(6)
t1 = perf_counter()
print(round(t1-t0, 4))

0
300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500
476.1664


In [27]:
mcs.possible_score_frequencies[6]

defaultdict(int,
            {DiceHand(1, score=100): 5317,
             DiceHand(1, 1, score=200): 2139,
             DiceHand(1, 1, 5, score=250): 1176,
             DiceHand(1, 5, score=150): 3316,
             DiceHand(5, score=50): 5308,
             DiceHand(5, 5, score=100): 2111,
             DiceHand(5, 5, 5, score=150): 510,
             DiceHand(5, 5, 5, score=500): 510,
             DiceHand(1, 5, 5, score=200): 1166,
             DiceHand(1, 1, 2, 2, 2, 5, score=450): 10,
             DiceHand(1, 1, 2, 2, 2, score=400): 46,
             DiceHand(1, 2, 2, 2, 5, score=350): 88,
             DiceHand(1, 2, 2, 2, score=300): 239,
             DiceHand(5, 2, 2, 2, score=250): 233,
             DiceHand(2, 2, 2, score=200): 480,
             DiceHand(1, 1, 1, score=300): 549,
             DiceHand(1, 1, 1, 5, score=350): 240,
             DiceHand(5, 1, 1, 1, score=1050): 240,
             DiceHand(1, 1, 1, score=1000): 549,
             DiceHand(1, 6, 6, 6, score=700): 230,
   

In [13]:
mcs.roll(3)

In [14]:
mcs.possible_score_frequencies[3]

defaultdict(int, {DiceHand(1, score=100): 1})

In [15]:
for i in range(4):
    mcs.roll(3)

In [16]:
mcs.possible_score_frequencies[3]

defaultdict(int,
            {DiceHand(1, score=100): 1, DiceHand(5, score=50): 3, None: 1})

In [4]:
class MonteCarloDiceHand(go.DiceHand):
    def __init__(self, num_dice):
        super().__init__(num_dice=num_dice)

        self.possible_score_frequencies = defaultdict(int)
        self.roll_obs = 0
        self.possible_score_obs = 0

    def roll(self):
        super().roll()
        self.roll_obs += 1
        # save resulting possible scores if any otherwise None
        if self.possible_scores():
            for dh in self.possible_scores():
                dh: go.DiceHand
                self.possible_score_obs += 1
                self.possible_score_frequencies[dh] += 1
        else:
            self.possible_score_frequencies[None] += 1

    @property
    def E_any_score(self):
        return 1 - (self.possible_score_frequencies[None] / self.roll_obs)

    @property
    def E_score_arithmetic(self):
        E_score = 0
        for dh, freq in self.possible_score_frequencies.items():
            if dh: E_score += dh.score * freq
        return (E_score / self.possible_score_obs) * self.E_any_score

In [5]:
mcdh = MonteCarloDiceHand(3)

In [6]:
mcdh

DiceHand(4, 4, 4, score=0)

In [7]:
mcdh.roll()

In [8]:
mcdh

DiceHand(1, 2, 2, score=0)

In [9]:
mcdh.possible_score_frequencies

defaultdict(int, {DiceHand(1, score=100): 1})

In [10]:
for i in range(4):
    mcdh.roll()

In [11]:
mcdh.possible_score_frequencies

defaultdict(int,
            {DiceHand(1, score=100): 1,
             None: 3,
             DiceHand(5, score=50): 1,
             DiceHand(5, 5, score=100): 1})

In [13]:
mcdh.roll_obs

5

In [14]:
mcdh.E_any_score

0.4

In [15]:
mcdh.E_score_arithmetic

33.333333333333336